## DATA PREPROCESSING

These are the most common data that we are going to use in TensorFlow (Deep Learning)

* Tables
* Images
* Text

 ### Preparing Tabular Data for Training

It is important to identify which columns are considered categorical. Neural Networks and other models of Machine Learning should prefer numeric inputs (One Hot Encoder)
Another aspect to take account of is the potential of interactions among multiple features


🤬 Sick of trying to use Titanic as an example. <br>
🤔 There are lot's of dataset out there ¿? Bigger ones that can teach us better. <br>
But I am still almost a newbie so I'll keep track of what books tell us to do

In [1]:
# Loading libraries
import functools
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
# Load data from Google's public storage
TRAIN_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
TEST_DATA_URL = 'https://storage.googleapis.com/tf-datasets/titanic/eval.csv'

train_file_path = tf.keras.utils.get_file('train.csv',TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file('eval.csv',TEST_DATA_URL)


In [3]:
print(train_file_path)

/root/.keras/datasets/train.csv


**📑 When datasets are bigger is a best prectice in TensorFlow to convert your table into a straming dataset.** <br>
This ensures that memory consumption wil be not affected by data size

In [4]:
train_df = pd.read_csv(train_file_path, header='infer')
test_df = pd.read_csv(test_file_path, header='infer')

In [5]:
train_df

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
...,...,...,...,...,...,...,...,...,...,...
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
623,0,male,25.0,0,0,7.0500,Third,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y
625,0,female,28.0,1,2,23.4500,Third,unknown,Southampton,n


In [6]:
# We can, with TensorFlow, set the target, the column for prediction, the label

LABEL_COLUMN = 'survived'
LABELS = [0,1]

train_ds = tf.data.experimental.make_csv_dataset(
    train_file_path,
    batch_size=3,
    label_name=LABEL_COLUMN,
    na_value='?',
    num_epochs=1)
train_ds = train_ds.ignore_errors()

test_ds = tf.data.experimental.make_csv_dataset(
    test_file_path,
    batch_size=3,
    label_name=LABEL_COLUMN,
    na_value='?',
    num_epochs=1)
test_ds = test_ds.ignore_errors()


In [7]:
# Inspect the data
for batch, label in train_ds.take(1):
  print(label)
  for key, value in batch.items():
    print('{}: {}'.format(key,value.numpy()))

tf.Tensor([0 1 1], shape=(3,), dtype=int32)
sex: [b'male' b'male' b'female']
age: [28. 28. 18.]
n_siblings_spouses: [0 0 0]
parch: [0 0 2]
fare: [ 7.8958 56.4958 13.    ]
class: [b'Third' b'Third' b'Second']
deck: [b'unknown' b'unknown' b'unknown']
embark_town: [b'Southampton' b'Southampton' b'Southampton']
alone: [b'y' b'y' b'n']


#### Major steps for training a paradigm to consume your training dataset:

* Designate columns by features type
* Decide wether or not to embed or cross columns
* Choose the columns of interest, possibly as an experiment
* Create a 'feature layer' for consumption by the training paradigm

##### Features types
Four numeric columns: 'age', 'n_siblings_spouses', 'parch', 'fares' <br>
Five categorical columns: 'sex','class','deck', 'embark_town', 'alone'

In [8]:
feature_columns = []

# numeric cols
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
  feature_columns.append(layers.Input(shape=(1,), name=header))

# other feature columns (categorical, etc.) could be added here

In [9]:
train_df.describe()

,survived,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000,627.000000
mean,0.387560,29.631308,0.545455,0.379585,34.385399
std,0.487582,12.511818,1.151090,0.792999,54.597730
min,0.000000,0.750000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,0.000000,7.895800
50%,0.000000,28.000000,0.000000,0.000000,15.045800
75%,1.000000,35.000000,1.000000,0.000000,31.387500
max,1.000000,80.000000,8.000000,5.000000,512.329200


In [10]:
age = feature_column.numeric_column('age')
age_input = layers.Input(shape=(1,), name='age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [11]:
h = {}

for col in train_df:
  if col in ['sex','class','deck', 'embark_town', 'alone']:
    print(col, ':', train_df[col].unique())
    h[col] = train_df[col].unique()

sex : ['male' 'female']
class : ['Third' 'First' 'Second']
deck : ['unknown' 'C' 'G' 'A' 'B' 'D' 'F' 'E']
embark_town : ['Southampton' 'Cherbourg' 'Queenstown' 'unknown']
alone : ['n' 'y']


In [12]:
sex_type = feature_column.categorical_column_with_vocabulary_list('Type', h.get('sex').tolist())
sex_type_one_hot = feature_column.indicator_column(sex_type)

class_type = feature_column.categorical_column_with_vocabulary_list('Type', h.get('class').tolist())
class_type_one_hot = feature_column.indicator_column(class_type)

deck_type = feature_column.categorical_column_with_vocabulary_list('Type', h.get('deck').tolist())
deck_type_one_hot = feature_column.indicator_column(deck_type)

embark_town_type = feature_column.categorical_column_with_vocabulary_list('Type', h.get('embark_town').tolist())
embark_town_type_one_hot = feature_column.indicator_column(embark_town_type)

alone_type = feature_column.categorical_column_with_vocabulary_list('Type', h.get('alone').tolist())
alone_type_one_hot = feature_column.indicator_column(alone_type)


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [13]:
deck = feature_column.categorical_column_with_vocabulary_list('deck',train_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [14]:
class_hashed = feature_column.categorical_column_with_hash_bucket('class', hash_bucket_size=4)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


##### Encoding Column Interactions as Possible Features

Sometimes intuition, experience and knowledge allows us to create new features by finding interactions between existing features

In [15]:
cross_type_feature = feature_column.crossed_column(['sex','class'], hash_bucket_size=5)

Instructions for updating:
Use `tf.keras.layers.experimental.preprocessing.HashedCrossing` instead for feature crossing when preprocessing data to train a Keras model.


In [16]:
# Start appending modified data
feature_columns = []

# Append numeric columns
for header in ['age', 'n_siblings_spouses', 'parch', 'fare']:
  feature_columns.append(feature_column.numeric_column(header))

# Append bucketized columns
age = feature_column.numeric_column('age')
age_buckets = feature_column.bucketized_column(age, boundaries=[23, 28, 35])
feature_columns.append(age_buckets)



In [17]:
# indicator_columns
indicator_column_names = ['sex', 'class', 'deck', 'embark_town', 'alone']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, train_df[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [18]:
# Append embedding columns
deck = feature_column.categorical_column_with_vocabulary_list(
    'deck', train_df.deck.unique())
deck_embedding = feature_column.embedding_column(deck, dimension=3)
feature_columns.append(deck_embedding)

In [19]:
# Append crossed columns
feature_columns.append(feature_column.indicator_column(cross_type_feature))

In [20]:
# Now create a feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

This layer will serve as the first input of the model we are going to build and train. This is how you'll provide all the feature engineering framewroks for the model's training process

##### Creating a Cross-Validation Dataset

We need to create a small set of our data for cross-validation purposes.<br>
(training, validation, testing)

In [21]:
val_df, test_df = train_test_split(test_df, test_size = 0.4)

In [24]:
batch_size = 33
labels = train_df.pop('survived')
working_ds = tf.data.Dataset.from_tensor_slices((dict(train_df), labels))
working_ds = working_ds.shuffle(buffer_size=len(train_df))
train_ds = working_ds.batch(batch_size)

In [25]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def pandas_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('survived')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [26]:
val_ds = pandas_to_dataset(val_df, shuffle=False, batch_size=batch_size)
test_ds = pandas_to_dataset(test_df, shuffle=False, batch_size=batch_size)

In [31]:
 # 1. Build the model
 model=tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dropout(.1),
    layers.Dense(1)])

 model.compile(optimizer='adam',
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               metrics=['accuracy'])

 model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100


13/19 [===================>..........] - ETA: 0s - loss: 0.9983 - accuracy: 0.6224 

19/19 [==============================] - 3s 29ms/step - loss: 0.9460 - accuracy: 0.6252 - val_loss: 0.6330 - val_accuracy: 0.6899
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.9116 - accuracy: 0.6619 - val_loss: 0.6890 - val_accuracy: 0.6835
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.6898 - accuracy: 0.6874 - val_loss: 0.5638 - val_accuracy: 0.7152
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 0.6290 - accuracy: 0.7081 - val_loss: 0.5798 - val_accuracy: 0.7278
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 0.6394 - accuracy: 0.6778 - val_loss: 0.5362 - val_accuracy: 0.6899
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 0.7036 - accuracy: 0.6715 - val_loss: 0.5738 - val_accuracy: 0.7342
Epoch 7/100
19/19 [==============================] - 0s 5ms/step - loss: 0.5709 - accuracy: 0.7305 - val_loss: 0.5089 - val_accuracy: 0.7532
Epoch 8/100
19/19 [=====